In [1]:
import os
import datetime

from vllm.steer_vectors.request import SteerVectorRequest
from vllm import LLM, SamplingParams

from easysteer.steer import StatisticalControlVector

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-13 21:51:17 [__init__.py:244] Automatically detected platform cuda.


In [2]:
model_path = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
# vector_path="/home/meixinyu/EasySteer/vectors/thinking_switch_pca_MATH-500.gguf" #change to your path
vector_path="MATH500.gguf" #change to your path

control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([-0.04544575,  0.01361476,  0.01698133, ...,  0.0365207 ,
         0.0359213 ,  0.00463693], shape=(1536,), dtype=float32), 1: memmap([-0.03687524,  0.01788449, -0.00667726, ...,  0.03739359,
         0.01404337,  0.0170301 ], shape=(1536,), dtype=float32), 2: memmap([-0.03864482, -0.00628719, -0.02879705, ...,  0.02464085,
         0.02684357,  0.01252755], shape=(1536,), dtype=float32), 3: memmap([ 0.02786397,  0.00761229,  0.01361889, ..., -0.00188065,
        -0.04521463, -0.01735996], shape=(1536,), dtype=float32), 4: memmap([ 0.03038106, -0.00119581,  0.00820935, ...,  0.01588252,
        -0.02874248, -0.01130662], shape=(1536,), dtype=float32), 5: memmap([-0.04012285, -0.01570467, -0.00127555, ..., -0.0050538 ,
         0.02423198,  0.03660234], shape=(1536,), dtype=float32), 6: memmap([-0.04255282, -0.02186516,  0.01580931, ..., -0.00768411,
         0.01432823,  0.01873141], shape=(1536,), dtype

In [3]:
steer_vector_request_pos = SteerVectorRequest(
            steer_vector_name="fast",
            steer_vector_id=1,
            steer_vector_local_path=vector_path,
            scale=8.0,
            target_layers=[19,28],
            prefill_trigger_tokens=[-1],
            prefill_trigger_positions=[-1],
            generate_trigger_tokens=[-1],
            debug=False,
            algorithm='direct'
        )
steer_vector_request_neg = SteerVectorRequest(
            steer_vector_name="slow",
            steer_vector_id=2,
            steer_vector_local_path=vector_path,
            scale=-8.0,
            target_layers=[19,28],
            prefill_trigger_tokens=[-1],
            prefill_trigger_positions=[-1],
            generate_trigger_tokens=[-1],
            debug=False,
            algorithm='direct'
        )

In [4]:
sampling_params = SamplingParams(temperature=0.0,max_tokens=4096)
prompt_template = "<|User|>Return your final response within \\boxed{}.\n%s<|Assistant|><think>\n"
prompt = "Find the constant term in the expansion of $$\\left(10x^3-\\frac{1}{2x^2}\\right)^{5}$$"
# prompt = "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."
input = prompt_template % prompt

In [5]:
llm = LLM(model=model_path, enable_steer_vector=True, tensor_parallel_size=1,enforce_eager=True)
output_base = llm.generate(
    input,
    sampling_params
)
generated_text_base = output_base[0].outputs[0].text

output_pos = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_pos
            )
generated_text_pos = output_pos[0].outputs[0].text

output_neg = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_neg
            )
generated_text_neg = output_neg[0].outputs[0].text

INFO 07-13 21:51:26 [config.py:841] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 07-13 21:51:26 [config.py:1472] Using max model len 131072
WARNING 07-13 21:51:26 [arg_utils.py:1770] --enable-steer-vector is not supported by the V1 Engine. Falling back to V0. 
WARNING 07-13 21:51:26 [arg_utils.py:1577] The model has a long context length (131072). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.


2025-07-13 21:51:26,791	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 07-13 21:51:26 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-13 21:51:26 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, r

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.11s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.11s/it]


INFO 07-13 21:51:30 [default_loader.py:272] Loading weights took 1.26 seconds


INFO 07-13 21:51:31 [model_runner.py:1255] Model loading took 3.3461 GiB and 1.437549 seconds
INFO 07-13 21:51:35 [worker.py:295] Memory profiling takes 3.98 seconds
INFO 07-13 21:51:35 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-13 21:51:35 [worker.py:295] model weights take 3.35GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 8.10GiB; the rest of the memory reserved for KV Cache is 31.17GiB.
INFO 07-13 21:51:35 [executor_base.py:115] # cuda blocks: 72954, # CPU blocks: 9362
INFO 07-13 21:51:35 [executor_base.py:120] Maximum concurrency for 131072 tokens per request: 8.91x
INFO 07-13 21:51:38 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 7.81 seconds


Processed prompts: 100%|██████████| 1/1 [00:54<00:00, 54.10s/it, est. speed input: 1.02 toks/s, output: 38.58 toks/s]


In [6]:
print(generated_text_base)

Okay, so I have this problem here: I need to find the constant term in the expansion of \(\left(10x^3 - \frac{1}{2x^2}\right)^5\). Hmm, let me think about how to approach this.

First, I remember that when you have an expression raised to a power, like \((a + b)^n\), you can use the binomial theorem to expand it. The binomial theorem says that each term in the expansion is of the form \(\binom{n}{k} a^{n - k} b^k\), where \(k\) ranges from 0 to \(n\). So, in this case, \(a\) is \(10x^3\), \(b\) is \(-\frac{1}{2x^2}\), and \(n\) is 5.

Alright, so the general term in the expansion will be \(\binom{5}{k} (10x^3)^{5 - k} \left(-\frac{1}{2x^2}\right)^k\). I need to find the term where the exponent of \(x\) is zero because that's the constant term. So, let me write that down.

First, let's compute the exponent of \(x\) in each term. The exponent from \((10x^3)^{5 - k}\) is \(3(5 - k)\), and the exponent from \(\left(-\frac{1}{2x^2}\right)^k\) is \(-2k\). So, the total exponent of \(x\) in e

In [7]:
print(generated_text_pos)

Okay, so I have this problem here: I need to find the constant term in the expansion of \(\left(10x^3 - \frac{1}{2x^2}\right)^5\). Hmm, let me think about how to approach this.

First, I remember that when you have an expression raised to a power, like \((a + b)^n\), you can use the binomial theorem to expand it. The binomial theorem says that each term in the expansion is of the form \(\binom{n}{k} a^{n - k} b^k\), where \(k\) ranges from 0 to \(n\). So, in this case, \(a\) is \(10x^3\), \(b\) is \(-\frac{1}{2x^2}\), and \(n\) is 5.

Alright, so the general term in the expansion will be \(\binom{5}{k} (10x^3)^{5 - k} \left(-\frac{1}{2x^2}\right)^k\). I need to find the term where the exponent of \(x\) is zero because that's the constant term. So, let me write that down.

First, let's compute the exponent of \(x\) in each term. The term \((10x^3)^{5 - k}\) will contribute \(3(5 - k)\) to the exponent of \(x\). Similarly, the term \(\left(-\frac{1}{2x^2}\right)^k\) will contribute \(-2k

In [8]:
print(generated_text_neg)

Okay, so I have this problem here: I need to find the constant term in the expansion of \(\left(10x^3 - \frac{1}{2x^2}\right)^5\). Hmm, let me think about how to approach this.

First, I remember that when you have an expression raised to a power, like \((a + b)^n\), you can use the binomial theorem to expand it. The binomial theorem says that each term in the expansion is of the form \(\binom{n}{k} a^{n - k} b^k\), where \(k\) ranges from 0 to \(n\). So, in this case, \(a\) is \(10x^3\), \(b\) is \(-\frac{1}{2x^2}\), and \(n\) is 5.

Alright, so I need to find the term where the exponent of \(x\) is zero because that's the constant term. That means I need to figure out for which value of \(k\) the exponent of \(x\) in the term \(\binom{5}{k} (10x^3)^{5 - k} \left(-\frac{1}{2x^2}\right)^k\) is zero.

Let me write that out. The exponent of \(x\) in each term is given by the exponents from both \(a\) and \(b\). So, for \(a^{5 - k}\), the exponent is \(3(5 - k)\), and for \(b^k\), the exp

In [9]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename=f'fractreason_GSM8K_{timestamp}.txt'

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"
)
# filename=f'../../temp/fractreason_GSM8K_{timestamp}.txt'
with open(filename, 'w', encoding='utf-8') as f:
    f.write("=== Base ===\n")
    f.write(generated_text_base + "\n\n")
    f.write("=== Positive: Fast thinking ===\n")
    f.write(generated_text_pos + "\n\n")
    f.write("=== Negative: Slow thinking ===\n")
    f.write(generated_text_neg + "\n")
    print("Seal tokens: ", len(tokenizer.tokenize(output_base[0].outputs[0].text, add_special_tokens=True)))
    print("Seal tokens: ", len(tokenizer.tokenize(output_pos[0].outputs[0].text, add_special_tokens=True)))
    print("Seal tokens: ", len(tokenizer.tokenize(output_neg[0].outputs[0].text, add_special_tokens=True)))

Seal tokens:  3994
Seal tokens:  2809
Seal tokens:  2087
